In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
import copy
from screen_utils import *
from cv_utils import *

In [2]:
model = load_model()

In [40]:
def kill_cows(model, fps, control_time, initial_wait=1.5, click_delay=5.0):
    time.sleep(initial_wait)
    
    window_coords = get_window_coords()
    width = window_coords[2] - window_coords[0]
    height = window_coords[3] - window_coords[1]
    window_center = (width // 2, height // 2)
    tracker = ObjectTracker(300, 3000, time_threshold=fps, n_hist=fps*2)
    
    click_timer = time.time() - 15
    
    target_id = -999
    for i in range(control_time * fps):
        prev_time = time.time()
        window_pixels = grab_area(*window_coords)
        mask = get_cow_mask(window_pixels)
        bounding_boxes = get_bounding_boxes(mask, pixels=window_pixels, model=model)
        objects = tracker.add_frame(bounding_boxes)
        
        # Get bounding boxes of confirmed objects
        obj_centers = set(objects.values())
        select_boxes = []
        
        for bounding_box in bounding_boxes:
            if bounding_box[0] in obj_centers:
                select_boxes.append(bounding_box)
                x, y, width, height = bounding_box[1]
                cropped_img = window_pixels[y:y+height, x:x+width]
                
        if time.time() - click_timer >= click_delay and target_id in objects.keys():
            print('Cannot retarget because how still alive')
        
        if time.time() - click_timer >= click_delay and target_id not in objects.keys():
            click_timer = time.time()
            if objects:
                sorted_objects = list(sorted(objects, key=lambda k: dist(objects[k], window_center)))
                selected_obj = sorted_objects[len(sorted_objects)//2]
                obj_center = objects[selected_obj]
                click(obj_center[0], obj_center[1],window_offset=(window_coords[0], window_coords[1]))

        while time.time() - prev_time < 1./fps:
            time.sleep(0.001)

In [43]:
kill_cows(model, 15, 999999, initial_wait=1.5, click_delay=27.0)

KeyboardInterrupt: 

In [21]:
def gen_cv_video(video_name, fps, video_len, initial_wait=1.5):
    time.sleep(initial_wait)
    
    window_coords = get_window_coords()
    width = window_coords[2] - window_coords[0]
    height = window_coords[3] - window_coords[1]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_name + '.mp4', fourcc, fps, (width*2,height*2))

    for _ in range(video_len * fps):
        prev_time = time.time()
        window_pixels = grab_area(*window_coords)
        mask = get_cow_mask(window_pixels)
        bounding_boxes = get_bounding_boxes(mask)
        img_copy = copy.deepcopy(window_pixels)
        draw_boxes(img_copy, bounding_boxes)
        img_with_mask = cv2.bitwise_or(window_pixels, window_pixels, mask=mask)
        mask_img = mask_to_img(mask)

        row_1 = np.concatenate([window_pixels, mask_img], axis=0)
        row_2 = np.concatenate([img_with_mask, img_copy], axis=0)
        full_display = np.concatenate([row_1, row_2], axis=1)

        full_display = cv2.cvtColor(full_display, cv2.COLOR_RGB2BGR)
        video_writer.write(full_display)

        while time.time() - prev_time < 1./fps:
            time.sleep(0.001)

    video_writer.release()
    
def gen_cv_video_v2(video_name, fps, video_len, frame_skip, initial_wait=1.5):
    time.sleep(initial_wait)
    
    window_coords = get_window_coords()
    width = window_coords[2] - window_coords[0]
    height = window_coords[3] - window_coords[1]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_name + '.mp4', fourcc, fps, (width*2,height*2))
    font = cv2.FONT_HERSHEY_SIMPLEX
    tracker = ObjectTracker(300, 3000, time_threshold=fps//2, n_hist=fps)

    for i in range(video_len * fps):
        prev_time = time.time()
        window_pixels = grab_area(*window_coords)
        mask = get_cow_mask(window_pixels)
        mask_img = mask_to_img(mask)
        
        if i % frame_skip == 0:
            bounding_boxes = get_bounding_boxes(mask)
            objects = tracker.add_frame(bounding_boxes)
        
        img_copy_1 = copy.deepcopy(window_pixels)
        img_copy_2 = copy.deepcopy(window_pixels)
        
        for obj_id, center in objects.items():
            bottom_left = tuple(np.array(center, dtype=np.int32) + np.array([-10, 10]))
            cv2.putText(img_copy_2, 'id: ' + str(obj_id), bottom_left,
                        font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

        obj_centers = set(objects.values())
        select_boxes = []
        for bounding_box in bounding_boxes:
            if bounding_box[0] in obj_centers:
                select_boxes.append(bounding_box)

        draw_boxes(img_copy_1, bounding_boxes)
        draw_boxes(img_copy_2, select_boxes)

        column_1 = np.concatenate([window_pixels, img_copy_1], axis=0)
        column_2 = np.concatenate([mask_img, img_copy_2], axis=0)
        full_display = np.concatenate([column_1, column_2], axis=1)

        full_display = cv2.cvtColor(full_display, cv2.COLOR_RGB2BGR)
        video_writer.write(full_display)

        while time.time() - prev_time < 1./fps:
            print('SDFJSDJFOIJSDOIFJOSID')
            time.sleep(0.001)

    video_writer.release()
    
def gen_cv_video_v3(video_name, fps, video_len, frame_skip, model, initial_wait=1.5):
    time.sleep(initial_wait)
    
    window_coords = get_window_coords()
    width = window_coords[2] - window_coords[0]
    height = window_coords[3] - window_coords[1]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_name + '.mp4', fourcc, fps, (width*2,height*2))
    font = cv2.FONT_HERSHEY_SIMPLEX
    tracker = ObjectTracker(300, 3000, time_threshold=fps//2, n_hist=fps)

    frames = []
    for i in range(video_len * fps):
        prev_time = time.time()
        window_pixels = grab_area(*window_coords)
        mask = get_cow_mask(window_pixels)
        mask_img = mask_to_img(mask)
        
        if i % frame_skip == 0:
            bounding_boxes = get_bounding_boxes(mask, pixels=window_pixels, model=model)
            bounding_boxes_2 = get_bounding_boxes(mask)
        
        img_copy_1 = copy.deepcopy(window_pixels)
        img_copy_2 = copy.deepcopy(window_pixels)

        draw_boxes(img_copy_1, bounding_boxes_2)
        draw_boxes(img_copy_2, bounding_boxes)

        column_1 = np.concatenate([window_pixels, img_copy_1], axis=0)
        column_2 = np.concatenate([mask_img, img_copy_2], axis=0)
        full_display = np.concatenate([column_1, column_2], axis=1)

        full_display = cv2.cvtColor(full_display, cv2.COLOR_RGB2BGR)
        frames.append(full_display)
        
        while time.time() - prev_time < 1./fps:
            time.sleep(0.001)

    for frame in frames:
        video_writer.write(frame)
        
    video_writer.release()

In [22]:
FPS = 13
VIDEO_LEN = 10 # Seconds
FRAME_SKIP = 1
gen_cv_video_v3('cow_detection_test', FPS, VIDEO_LEN, FRAME_SKIP, model, initial_wait=1.5)

In [20]:
# FPS = 20
# VIDEO_LEN = 5 # Seconds
# FRAME_SKIP = 2
# gen_cv_video_v2('tracking_test', FPS, VIDEO_LEN, FRAME_SKIP, initial_wait=1.5)

In [80]:
# FPS = 20
# VIDEO_LEN = 10 # Seconds
# gen_cv_video('test_2', 20, 10)